In [1]:
# Install transformers from source - only needed for versions <= v4.34

import torch
from transformers import pipeline

pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install accelerate`

In [ ]:
messages = [
    {
        "role": "system",
        "content": "You are supposed act as an agent that evaluates any problem solution pair on a scale of 1-10 and give feedback. You must return a number grading the solution as well as one sentence feedback on how the solution can be improved",
    },
    {"role": "user", "content": '''

    Problem: The low-density polyethylene (LPDE) material used to make food packaging and shopping bags makes them unacceptable for recycling more than once.
    Solution: "Low-density polyethylene (LPDE) bags can only be recycled ONCE because they can interfere with sorting machinery.

The initiator used in the production of grocery and shopping bags is ORGANIC PEROXIDE, which poses a risk to human health after recycling. Because of all its harmful effects, this initiator is the primary reason for IMPOSSIBLE RECYCLING. When using organic peroxide in any form of product, a REPLACEMENT or RECYCLEABLE INITIATOR should be used.

MAKING OF LPDE:

Unreacted gas is first compressed by melting natural gas ethylene to form new first-compression gas. In the second compressor, this newly compressed gas is once again melted with unreacted gases. At this point, a catalyst (organic peroxide) and a second compressed gas are mixed into the reactor. Polymerization occurs at a particular pressure and temperature. The unreacted gases are then separated from the polymerization process to extract the polymers, and now they are finally pelletized. Thereafter, the pellets can be dried by injecting hot air. Now, these pellets are used to make plastic bags."

    '''

    },
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=512, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])


In [2]:
!pip install transformers
import transformers
from transformers import LlamaForCausalLM, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import torch
from transformers import pipeline
!pip install accelerate -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.5 MB/s eta 0:00:00


In [3]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = LlamaForCausalLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
#Data
import pandas as pd
import numpy as np
df = pd.read_json("/content/syntheticdata.json")

df['evaluation'] = df['evaluation'].apply(lambda x: ', '.join([f"{key}: {value}" for key, value in x.items()]))

train_texts = df.apply(lambda row: ' '.join(row), axis=1).tolist()

# Print the resulting array
print(train_texts)

['The construction industry is a significant contributor to global waste, producing approximately 1.3 billion tons annually. Utilizing modular and reusable building materials to minimize construction waste. economic_stability: 8, technological_innovation: 7, feasibility: 9, circular_economy: 8 Great solution! Modular construction not only reduces waste but also promotes economic stability and feasibility. Consider exploring even more innovative materials for better technological innovation.', 'The challenge of generating enough electrical power without using fossil fuels is a pressing issue. Investing in solar power farms and advancing energy storage technologies. economic_stability: 9, technological_innovation: 8, feasibility: 7, circular_economy: 6 Your focus on solar power is commendable, showing good economic stability and technological innovation. However, consider exploring circular economy principles for more sustainable solutions.', 'The increased carbon footprint from digital 

In [5]:
tokenized_train_texts = tokenizer(train_texts, truncation=True,
                                  padding=True, return_tensors='pt')
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,
                                                mlm=False)

In [6]:
training_args = TrainingArguments(
    output_dir="./output", overwrite_output_dir=True, num_train_epochs=3,
    per_device_train_batch_size=16, save_steps=10_000, save_total_limit=2,
)

trainer = Trainer(
    model=model, args=training_args, data_collator=data_collator,
    train_dataset=tokenized_train_texts
)

trainer.train()

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_train_texts
)

In [ ]:
trainer.train()

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
import gc
del data_collator
gc.collect()

0

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)


'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 4            |        cudaMalloc retries: 4         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   8320 KiB |  14603 MiB |  52567 MiB |  52559 MiB |\n|       from large pool |   8320 KiB |  14546 MiB |  52450 MiB |  52442 MiB |\n|       from small pool |      0 KiB |     57 MiB |    117 MiB |    117 MiB |\n|---------------------------------------------------------------------------|\n| Active memory         |   8320 KiB |  14603 MiB |  52567 MiB |  52559 MiB |\n|       from large pool |   8320 KiB |  14546 MiB |

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW

# Load pre-trained model and tokenizer
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # Specify the model name from Hugging Face Model Hub
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Assume your dataset is in a CSV file with 'problem', 'solution', and 'score' columns
# Create a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        problem_solution = self.data.iloc[idx]['problem'] + ' ' + self.data.iloc[idx]['solution']
        score = float(self.data.iloc[idx]['result'])
        return problem_solution, score


Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at TinyLlama/TinyLlama-1.1B-Chat-v1.0 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

# Load and split your dataset (assuming you have a pandas DataFrame)
# train_data, val_data, test_data = split_your_dataset_accordingly()
df = pd.read_json("/content/syntheticdata.json")
# df["input"] = "Problem: " + df["problem"] + " Solution: " + ["solution"]
df['evaluation'] = df['evaluation'].apply(lambda x: ', '.join([f"{key}: {value}" for key, value in x.items()]))
df["result"] = df["evaluation"] + df["feedback"]

df = df.drop(['feedback', 'evaluation'], axis =1)
df

In [ ]:
train_texts = df.fillna(0)
# Create DataLoader for training
train_texts
train_loader = DataLoader(train_texts, batch_size=8, shuffle=True)

# Training settings
num_epochs = 1
learning_rate = 2e-5
optimizer = AdamW(model.parameters(), lr=learning_rate)
print(train_texts)
# Training loop
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        inputs, labels = batch
        inputs = tokenizer(inputs, return_tensors="pt", padding=True, truncation=True)
        labels = torch.tensor(labels).unsqueeze(1)

        # Forward pass
        outputs = model(**inputs)
        loss = torch.nn.functional.mse_loss(outputs.logits, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

                                                input  \
0   Problem: The construction industry is a signif...   
1   Problem: The challenge of generating enough el...   
2   Problem: The increased carbon footprint from d...   
3   Problem: The fashion industry is a top contrib...   
4   Problem: Electronic devices are often replaced...   
5   Problem: Businesses worldwide expend substanti...   
6   Problem: More than 130 billion plastic bottles...   
7   Problem: In congested cities like Berlin, find...   
8   Problem: The surplus of plastic waste is a maj...   
9   Problem: The usage of plastic bottles is a wid...   
10  Problem: Despite the environmental principles ...   
11  Problem: The fashion industry is the second-la...   
12  Problem: The fashion industry has a considerab...   
13  Problem: The solution is meant to solve the is...   
14  Problem: Single-use plastic packaging has beco...   
15  Problem: The fashion industry contributes abou...   
16  Problem: Rapid advancement 

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


KeyError: 5

,input,result
0,Problem: The construction industry is a signif...,"economic_stability: 8, technological_innovatio..."
1,Problem: The challenge of generating enough el...,"economic_stability: 9, technological_innovatio..."
2,Problem: The increased carbon footprint from d...,"economic_stability: 7, technological_innovatio..."
3,Problem: The fashion industry is a top contrib...,"economic_stability: 8, technological_innovatio..."
4,Problem: Electronic devices are often replaced...,"economic_stability: 9, technological_innovatio..."
5,Problem: Businesses worldwide expend substanti...,"economic_stability: 7, technological_innovatio..."
6,Problem: More than 130 billion plastic bottles...,"economic_stability: 8, technological_innovatio..."
7,"Problem: In congested cities like Berlin, find...","economic_stability: 9, technological_innovatio..."
8,Problem: The surplus of plastic waste is a maj...,"economic_stability: 8, technological_innovatio..."
9,Problem: The usage of plastic bottles is a wid...,"economic_stability: 7, technological_innovatio..."
